In [6]:
# owsBaseURL='http://tapviewer.vgt.vito.be/geoserver/PV_MEP/ows'
owsBaseURL='http://mapcachedev.vgt.vito.be/geoserver/PV_MEP/ows'
wcsCredentials=('driesj', 'xxx')

In [7]:
import requests
from xml.etree import ElementTree

def getCoverages():

    payload = {
        'service': 'WCS',
        'request': 'GetCapabilities',
        'version': '2.0.1',
    }
    
    response = requests.get(owsBaseURL, params=payload)
    tree = ElementTree.fromstring(response.content)
    namespaces = {
        'wcs': 'http://www.opengis.net/wcs/1.1.1',
        'wcs2': 'http://www.opengis.net/wcs/2.0'
    }
    coverages = map(lambda x:x.text, tree.findall('wcs2:Contents/wcs2:CoverageSummary/wcs2:CoverageId',namespaces))
    return coverages

coverages = getCoverages()
print "\n".join(coverages)

PV_MEP__PROBAV_S5_TOC_100M_BITMASK
PV_MEP__PROBAV_S5_TOC_100M_BROWSE
PV_MEP__PROBAV_S5_TOC_100M_NDVI
PV_MEP__PROBAV_S10_TOC_1KM_BROWSE
PV_MEP__PROBAV_S10_TOC_1KM_GEOMETRY
PV_MEP__PROBAV_S10_TOC_1KM_NDVI
PV_MEP__PROBAV_S10_TOC_1KM_RADIOMETRY
PV_MEP__PROBAV_S10_TOC_333M_NDVI
PV_MEP__PROBAV_S1_TOA_1KM_BROWSE
PV_MEP__PROBAV_S1_TOA_1KM_NDVI
PV_MEP__PROBAV_S1_TOC_1KM_BROWSE
PV_MEP__PROBAV_S1_TOC_1KM_NDVI


In [18]:
import datetime
import matplotlib.pyplot as plt
from datetime import date

def pointWCSRequest(coverageId, lon, lat, startDate = None, endDate = None):
    
    wcsURL = owsBaseURL
    
    payload = {
        'SERVICE': 'WCS',
        'REQUEST': 'GetCoverage',
        'VERSION': '2.0.1',
        'coverageId': coverageId,
        'FORMAT': 'application/json',
        'subset': ['Long,http://www.opengis.net/def/crs/EPSG/0/4326('+str(lon)+')',
                   'Lat,http://www.opengis.net/def/crs/EPSG/0/4326('+str(lat)+')']
    }
        
    if startDate != None and endDate != None:
        timeSubset = 'time("'+startDate.strftime('%Y-%m-%dT%H:%M:%S.000Z')+'","'+endDate.strftime('%Y-%m-%dT%H:%M:%S.000Z')+'")'
        payload['subset'].append(timeSubset)    
    
    return requests.get(wcsURL, params=payload, auth=wcsCredentials)

def wcsTimeSeries(coverageID, lon, lat, startDate = None, endDate = None):

    response = pointWCSRequest(coverageID, lon, lat, startDate, endDate)
    timeSeries = response.json()['timeseries']
    timeSeries = map(lambda x:[datetime.datetime.fromtimestamp(x[0]/1000),x[1]], timeSeries)
    return timeSeries

def plotTimeSeries(timeSeries):
    timeAxis = map(lambda x:x[0], timeSeries)
    valueAxis = map(lambda x:x[1],timeSeries)
    plt.figure(figsize=(25,10))
    plt.plot(timeAxis, valueAxis, 'ro')
    # plt.xticks(timeAxis)
    # plt.grid(True)

lon,lat = 4.71213078915, 50.5839444717
startDate=date(2014, 9, 1)
endDate=date(2015, 5, 1)

ndviTS = wcsTimeSeries('PV_MEP__PROBAV_S5_TOC_100M_NDVI', lon, lat, startDate, endDate)
bitmaskTS = wcsTimeSeries('PV_MEP__PROBAV_S5_TOC_100M_BITMASK', lon, lat, startDate, endDate)

# filter nodata values
for i, ts in enumerate(ndviTS):
    if ts[1] == 255:
        del ndviTS[i]
        del bitmaskTS[i]

In [19]:
ndvi300TS = wcsTimeSeries('PV_MEP__PROBAV_S10_TOC_333M_NDVI', lon, lat, startDate, endDate)
bitmask300TS = wcsTimeSeries('PV_MEP__PROBAV_S10_TOC_333M_BITMASK', lon, lat, startDate, endDate)

In [9]:
import sys
sys.path
! pip install --user mpld3
if '/home/driesj/.local/lib/python2.6/site-packages/' not in sys.path:
    sys.path.append('/home/driesj/.local/lib/python2.6/site-packages/')
sys.path

DEPRECATION: Python 2.6 is no longer supported by the Python core team, please upgrade your Python. A future version of pip will drop support for Python 2.6


['',
 '/usr/hdp/current/spark-client/python',
 '/usr/hdp/current/spark-client/python/lib/py4j-0.9-src.zip',
 '/usr/local/lib/python27.zip',
 '/usr/local/lib/python2.7',
 '/usr/local/lib/python2.7/plat-linux2',
 '/usr/local/lib/python2.7/lib-tk',
 '/usr/local/lib/python2.7/lib-old',
 '/usr/local/lib/python2.7/lib-dynload',
 '/usr/local/lib/python2.7/site-packages',
 '/usr/local/lib/python2.7/site-packages/setuptools-28.8.0-py2.7.egg',
 '/usr/local/lib/python2.7/site-packages/pip-9.0.1-py2.7.egg',
 '/usr/local/lib/python2.7/site-packages/GDAL-1.11.2-py2.7-linux-x86_64.egg',
 '/usr/local/lib/python2.7/site-packages/IPython/extensions',
 '/home/driesj/.ipython',
 '/home/driesj/.local/lib/python2.6/site-packages/']

In [21]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
fig.set_size_inches(12, 4)
ndviTimeStamps = map(lambda x:x[0], ndviTS)
ndviValues = map(lambda x:(x[1]-20)/250, ndviTS)
bitmaskValues = map(lambda x:int(x[1])&7, bitmaskTS)
print bitmaskValues

cmp = matplotlib.colors.ListedColormap(
    [ 'green'  # clear (0)
     ,'red'    # shadow (1)
     ,'orange' # unknown (2)
     ,'grey'   # cloud (3)
     ,'blue'], # ice (4)
    5)

scatter = ax.scatter(ndviTimeStamps, ndviValues, c=bitmaskValues, s=30, cmap=cmp)
ndvi300TimeStamps = map(lambda x:x[0], ndvi300TS)
ndvi300Values = map(lambda x:(x[1]-20)/250, ndvi300TS)
bitmask300Values = map(lambda x:int(x[1])&7, bitmask300TS)
scatter300 = ax.scatter(ndvi300TimeStamps, ndvi300Values,c=bitmask300Values,marker='^', s=30, cmap=cmp)

import mpld3

css = '''
    table
    {
        border-spacing: 10px;
        border-collapse: separate;
        background-color: #cccccc;
    }
    td
    {
        background-color: #cccccc;
    }
    table, td
    {
        font-family: Arial, Helvetica, sans-serif;
        text-align: left;
    }
'''

labels = ['''<table>
             <tr>
               <td>NDVI:</td>
               <td>{0}</td>
             </tr>
             <tr>
               <td>Date:</td>
               <td>{1}</td>
             </tr>             
             <tr>
               <td>Bitmask:</td>
               <td>{2}</td>
             </tr>                          
           </table>'''
          .format((ts[1]-20)/250,ts[0].strftime('%d-%m-%Y'),int(bitmaskTS[index][1])) for index, ts in enumerate(ndviTS)]
# tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
tooltip = mpld3.plugins.PointHTMLTooltip(scatter, labels=labels, css=css)
mpld3.plugins.connect(fig, tooltip)

mpld3.display(fig)

[0, 3, 1, 3, 3, 3, 0, 3, 3, 3, 3, 3, 4, 0, 0, 3, 0, 3, 0, 3, 3, 3, 4, 4, 3, 3, 3, 3, 4, 3, 0, 0, 0, 3, 3, 0, 3, 0, 3, 4, 3, 3, 0, 3, 0, 0, 3, 3]
